### Imports

In [ ]:
import datetime
import os
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
#from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
#Setting the device for computation CPU/GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#random no for setting experiment
runID = str(random.randint(11111, 99999))
numberOfEpochs = 1#400
batchSize = 12
#Reading data
all_data_path = "./data/"
log_dir = os.path.join("./tensorboard/", runID)
use_saved_norm_constants = True
cultivar = "ColdHardiness_Grape_Merlot.csv"
cultivar_data_path = os.path.join(all_data_path, cultivar)
df = pd.read_csv(cultivar_data_path)

In [ ]:
features = [
    # 'DATE', # date of weather observation
    # 'AWN_STATION', # closest AWN station
    # 'SEASON',
    # 'SEASON_JDAY',
    # 'DORMANT_SEASON',
    # 'YEAR_JDAY',
    # 'PHENOLOGY',
    'PREDICTED_Hc',
    # 'PREDICTED_Budbreak',
    # mean temperature is the calculation of (max_f+min_f)/2 and then converted to Celsius. # they use this one
    'MEAN_AT',
    'MIN_AT',  # a
    # 'AVG_AT', # average temp is AgWeather Network
    'MAX_AT',  # a
    'MIN_REL_HUMIDITY',  # a
    'AVG_REL_HUMIDITY',  # a
    'MAX_REL_HUMIDITY',  # a
    'MIN_DEWPT',  # a
    'AVG_DEWPT',  # a
    'MAX_DEWPT',  # a
    'P_INCHES',  # precipitation # a
    'WS_MPH',  # wind speed. if no sensor then value will be na # a
    'MAX_WS_MPH',  # a
    # 'WD_DEGREE', # wind direction, if no sensor then value will be na
    # 'LW_UNITY', # leaf wetness sensor
    # 'SR_WM2', # solar radiation
    # 'MIN_ST2',
    # 'ST2',
    # 'MAX_ST2',
    # 'MIN_ST8',
    # 'ST8', # soil temperature
    # 'MAX_ST8',
    # 'SM8_PCNT', # soil moisture import matplotlib.pyplot as plt@ 8-inch depth # too many missing values for merlot
    # 'SWP8_KPA', # stem water potential @ 8-inch depth # too many missing values for merlot
    # 'MSLP_HPA', # barrometric pressure
    # 'ETO', # evaporation of soil water lost to atmosphere
    # 'ETR' # ???
]

label = ['LT10', 'LT50', 'LT90']
lt10_knn_path = "./knn/LT10_rnn.pkl"
lt50_knn_path = "./knn/LT50_rnn.pkl"
lt90_knn_path = "./knn/LT90_rnn.pkl"
budbreak_knn_path = "./knn/Budbreak_rnn.pkl"

### Data processing functions

In [ ]:
def linear_interp(x, y, missing_indicator, show=True):
    y = np.array(y)
    if (not np.isnan(missing_indicator)):
        missing = np.where(y == missing_indicator)[0]
        not_missing = np.where(y != missing_indicator)[0]
    else:
        # special case for nan values
        missing = np.argwhere(np.isnan(y)).flatten()
        all_idx = np.arange(0, y.shape[0])
        not_missing = np.setdiff1d(all_idx, missing)

    interp = np.interp(x, not_missing, y[not_missing])

    if show == True:
        plt.figure(figsize=(16, 9))
        plt.title("Linear Interp. result where missing = " +
                  str(missing_indicator) + "  Values replaced: " + str(len(missing)))
        plt.plot(x, interp)
        # plt.show()

    return interp


def remove_na(column_name):
    total_na = df[column_name].isna().sum()

    df[column_name] = df[column_name].replace(np.nan, -100)
    df[column_name] = linear_interp(
        np.arange(df.shape[0]), df[column_name], -100, False)
    if df[column_name].isna().sum() != 0:
        assert False

    print("Removed", total_na, "nan from", column_name)

    return


def get_before_after_phenology(max_season_len, season, df):
    #print(season[0], season[-1])

    _y = np.zeros((season_max_length, 1))
    _y[:] = np.NaN

    season_df = df["PHENOLOGY"].iloc[season[0]:season[-1]]

    budbreak_index = season_df.loc[season_df ==
                                   "Budburst/Budbreak"].index.values

    if (len(budbreak_index) == 1):
        budbreak_index = budbreak_index[-1] - season[0]
        #print("budbreak idx:",  budbreak_index)
        #print("_y before", list(_y.flatten()))

        _y[0:budbreak_index] = 0
        _y[budbreak_index:] = 1

        #print("_y after", list(_y.flatten()))
        #print("budbreak_index", _y[budbreak_index])

    return _y


def split_and_normalize(_df, max_season_len, seasons, x_mean, x_std):
    x = []
    y = []

    for i, season in enumerate(seasons):
        _x = (_df[features].loc[season, :]).to_numpy()

        _x = np.concatenate(
            (_x, np.zeros((season_max_length - len(season), len(features)))), axis=0)

        add_array = np.zeros((season_max_length - len(season), len(label)))
        add_array[:] = np.NaN

        _y = _df.loc[season, :][label].to_numpy()
        _y = np.concatenate((_y, add_array), axis=0)

        _phen = get_before_after_phenology(
            season_max_length, season, df)  # get before-after phen
        # set before-after phen as another output dim
        _y = np.concatenate((_y, _phen), axis=1)

        #_y = np.reshape(_y, (_y.shape[0], _y.shape[1], _y.shape[2], 1))

        x.append(_x)
        y.append(_y)

    x = np.array(x)
    y = np.array(y)

    norm_features_idx = np.arange(0, x_mean.shape[0])

    x[:, :, norm_features_idx] = (
        x[:, :, norm_features_idx] - x_mean) / x_std  # normalize

    return x, y

#For budbreak, gets % gt < pred (higher is better)
def _get_under(pred, gt):
    total = 0
    under = 0
    for i in range(gt.shape[0]):
        if (np.isnan(pred[i]) == False and np.isnan(gt[i]) == False):
            #print(pred[i], gt[i])
            total += 1

            if (gt[i] < pred[i]):
                under += 1

    print(under, total, under / total * 100)
    return under / total * 100

#For budbreak
def plot_budbreak(title, season, _y):
    season_idx = np.arange(0, len(season))

    add_array = np.zeros((season_max_length - len(season)))
    add_array[:] = np.NaN

    _y = np.concatenate((_y, add_array), axis=0)

    plt.title(title)

    plt.scatter(x=season_idx, y=_y[:len(season)], label="Prediction")

    _phen = get_before_after_phenology(
        252, season, df)  # get before-after phen

    # draw vertical red line marking true budbreak if it exists
    try:
        true_budbreak_idx = np.where(_phen == 1)[0][0]
        print("true_budbreak_idx:", true_budbreak_idx)
        print("predicted budbreak idx", np.where(_y > 0.5)[0][0])
        plt.vlines(x=true_budbreak_idx, ymin=0, ymax=5,
                   colors='red', label='True Budbreak')
    except:
        pass

    plt.xlabel("Season Day")
    plt.ylabel("Budbreak")

    plt.ylim(0, 1.1)

    plt.legend(loc="upper left")

    # plt.show()
    return

### Set up data for RNN training 

In [ ]:
for feature_col in features:  # remove nan and do linear interp.
    remove_na(feature_col)

if False:
    for f in features:
        plt.figure(figsize=(16, 9))
        plt.title(f)
        plt.plot(np.arange(df.shape[0]), df[f])
        plt.show()

seasons = []
last_x = 0
idx = -1
season_max_length = 0
for x in df[df["DORMANT_SEASON"] == 1].index.tolist():
    if x - last_x > 1:
        seasons.append([])
        if idx > -1:
            season_max_length = max(season_max_length, len(seasons[idx]))
        idx += 1
    seasons[idx].append(x)
    last_x = x

season_max_length = max(season_max_length, len(seasons[idx]))

del seasons[19]  # drop season 2007 to 2008 [252 days] because of gap in temperature data 

print("len(seasons)", len(seasons))
print("Season lengths", [len(x) for x in seasons])
print("Max season length", season_max_length)

if False:
    for i, season in enumerate(seasons):
        _start = str(df[["DATE"]].loc[season[0]].item().split("-")[0])
        _end = str(df[["DATE"]].loc[season[-1]].item().split("-")[0])
        print(str(i) + ".", _start + " to " + _end,
              "[" + str(len(season)) + " days]")
        #print("AVG_DEWPT", list(df["AVG_DEWPT"].iloc[season[0]:season[-1]] ))
        _lt50 = np.array(df["LT50"].iloc[season[0]:season[-1]])
        print("n LT50", np.count_nonzero(~np.isnan(_lt50)))
if (use_saved_norm_constants == True):
    try:
        with open('normalizing_constants.pkl', 'rb') as _f:
            norm_constants = pickle.load(_f)
            x_mean = norm_constants["x_mean"]
            x_std = norm_constants["x_std"]
            print("restored constants from normalizing_constants.pkl. Constants were generated for model ID",
                  norm_constants["runID"])
    except:
        print("normalizing_constants.pkl not found.")
        x_mean = df[features].mean().to_numpy()
        x_std = df[features].std().to_numpy()
else:
    print("Not using normalizing_constants.pkl")
    x_mean = df[features].mean().to_numpy()
    x_std = df[features].std().to_numpy()

print("Normalizing Constants:")
print("x_mean", x_mean)
print("x_std", x_std)

# if state pickle does not exist, create it and quit
if os.path.isfile('normalizing_constants.pkl') == False:
    with open('normalizing_constants.pkl', 'wb') as _f:
        norm_constants = {
            "runID": runID,
            "features": features,
            "x_mean": x_mean,
            "x_std": x_std
        }
        pickle.dump(norm_constants, _f)
        print("Saved normalizing constants to file.")

x_train, y_train = split_and_normalize(
    df, season_max_length, seasons[:-2 or None], x_mean, x_std)

def get_not_nan(y):
    return np.argwhere(np.isnan(y) == False)

print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)
print("length of sequence", len(x_train[0]))
print("length of sequence", len(y_train[0]))

x_test, y_test = split_and_normalize(
    df, season_max_length, seasons[-2:], x_mean, x_std)

print("shape of test dataset", x_test.shape)
print("shape of test dataset", y_test.shape)
print("length of sequence", len(x_test[0]))
print("length of sequence", len(y_test[0]))

### Network Architecture

In [ ]:
#For training the RNN
class net(nn.Module):  # LT50 and budbreak
    def __init__(self, input_size):
        super(net, self).__init__()

        self.numLayers = 1

        self.penul = 1024

        self.memory_size = 2048

        self.linear1 = nn.Linear(input_size, 1024)
        self.linear2 = nn.Linear(1024, 2048)
        self.rnn = nn.GRU(input_size=2048, hidden_size=self.memory_size,
                          num_layers=self.numLayers, batch_first=True)
        #self.dropout = nn.Dropout(p=0.1)
        self.linear3 = nn.Linear(self.memory_size, self.penul)  # penul
        self.linear4 = nn.Linear(self.penul, 1)  # LT10
        self.linear5 = nn.Linear(self.penul, 1)  # LT50
        self.linear6 = nn.Linear(self.penul, 1)  # LT90
        self.linear7 = nn.Linear(self.penul, 1)  # Budbreak

    def forward(self, x, h=None):
        batch_dim, time_dim, state_dim = x.shape

        out = self.linear1(x).relu()

        #out = self.dropout(out)

        out = self.linear2(out).relu()

        if h is None:
            h = torch.zeros(self.numLayers, batch_dim,
                            self.memory_size, device=x.device)

        out, h_next = self.rnn(out, h)  # rnn out

        out_s = self.linear3(out).relu()  # penul

        out_lt_10 = self.linear4(out_s)  # LT10
        out_lt_50 = self.linear5(out_s)  # LT50
        out_lt_90 = self.linear6(out_s)  # LT90

        out_ph = self.linear7(out_s).sigmoid()  # Budbreak

        # return out_s, out_lt, out_ph, h_next
        return out_lt_10, out_lt_50, out_lt_90, out_ph, h_next

#For doing K-nearest neighbor search using RNN predictions (same weights but different output
class knn_net(nn.Module): # LT50 and budbreak
    def __init__(self, input_size):
        super(knn_net, self).__init__()

        self.numLayers = 1
        
        self.penul = 1024
        
        self.memory_size = 2048

        self.linear1 = nn.Linear(input_size, 1024)
        self.linear2 = nn.Linear(1024, 2048)
        self.rnn = nn.GRU(input_size=2048, hidden_size=self.memory_size, num_layers=self.numLayers, batch_first=True)
        #self.dropout = nn.Dropout(p=0.1)
        self.linear3 = nn.Linear(self.memory_size, self.penul) # penul
        self.linear4 = nn.Linear(self.penul, 1) # LT10
        self.linear5 = nn.Linear(self.penul, 1) # LT50
        self.linear6 = nn.Linear(self.penul, 1) # LT90
        self.linear7 = nn.Linear(self.penul, 1) # Budbreak
        
    def forward(self, x, h=None):
        batch_dim, time_dim, state_dim = x.shape

        out = self.linear1(x).relu()
        
        #out = self.dropout(out)
    
        out = self.linear2(out).relu()

        if h is None:
            h = torch.zeros(self.numLayers, batch_dim, self.memory_size, device=x.device)

        out, h_next = self.rnn(out, h) # rnn out
        
        out_s = self.linear3(out).relu() # penul

        out_lt_10 = self.linear4(out_s) # LT10
        out_lt_50 = self.linear5(out_s) # LT50
        out_lt_90 = self.linear6(out_s) # LT90

        out_ph = self.linear7(out_s).sigmoid() # Budbreak
        
        #return out_lt_10, out_lt_50, out_lt_90, out_ph, h_next
        return out_s, out

### Set up Training

In [ ]:
model = net(np.array(x_train).shape[-1])
model.to(device)
trainable_params = sum([np.prod(p.size()) for p in filter(
    lambda p: p.requires_grad, model.parameters())])
print("Trainable Parameters:", trainable_params)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
criterion.to(device)
criterion2 = nn.BCELoss()
criterion2.to(device)
writer = SummaryWriter(log_dir)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape, y_train.shape)

train_dataset = TensorDataset(x_train, y_train)
trainLoader = DataLoader(train_dataset, batch_size=batchSize)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(y_test)

print(x_test.shape, y_test.shape)

val_dataset = TensorDataset(x_test, y_test)
valLoader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False)

### Training Loop

In [ ]:
for epoch in range(numberOfEpochs):

    # Training Loop
    with tqdm(trainLoader, unit="batch") as tepoch:
        model.train()

        tepoch.set_description(f"Epoch {epoch + 1}/{numberOfEpochs} [T]")
        total_loss = 0
        count = 0
        for i, (x, y) in enumerate(trainLoader):
            x_torch = x.to(device)
            y_torch = y.to(device)

            count += 1

            out_lt_10, out_lt_50, out_lt_90, out_ph, _ = model(x_torch)

            optimizer.zero_grad()       # zero the parameter gradients
            
            n_nan = get_not_nan(y[:, :, 0])  # LT10/50/90 not NAN
            loss_lt_10 = criterion(
                out_lt_10[n_nan[0], n_nan[1]], y_torch[:, :, 0][n_nan[0], n_nan[1]][:, None])  # LT10 GT

            n_nan = get_not_nan(y[:, :, 1])  # LT10/50/90 not NAN
            loss_lt_50 = criterion(
                out_lt_50[n_nan[0], n_nan[1]], y_torch[:, :, 1][n_nan[0], n_nan[1]][:, None])  # LT50 GT

            n_nan = get_not_nan(y[:, :, 2])  # LT10/50/90 not NAN
            loss_lt_90 = criterion(
                out_lt_90[n_nan[0], n_nan[1]], y_torch[:, :, 2][n_nan[0], n_nan[1]][:, None])  # LT90 GT

            n_nan = get_not_nan(y[:, :, 3])  # budbreak not NAN
            loss_ph = criterion2(
                out_ph[n_nan[0], n_nan[1]], y_torch[:, :, 3][n_nan[0], n_nan[1]][:, None])  # Budbreak GT

            loss = loss_lt_10 + loss_lt_50 + loss_lt_90 + loss_ph

            loss.backward()             # backward +
            optimizer.step()            # optimize

            total_loss += loss.item()

            tepoch.set_postfix(Train_Loss=total_loss / count)
            tepoch.update(1)

        writer.add_scalar('Train_Loss', total_loss / count, epoch)

    # Validation Loop
    with torch.no_grad():
        with tqdm(valLoader, unit="batch") as tepoch:

            model.eval()

            tepoch.set_description(f"Epoch {epoch + 1}/{numberOfEpochs} [V]")
            total_loss = 0
            count = 0
            for i, (x, y) in enumerate(valLoader):
                x_torch = x.to(device)
                y_torch = y.to(device)
                count += 1
                out_lt_10, out_lt_50, out_lt_90, out_ph, _ = model(x_torch)
                #getting non nan values is slow right now due to copying to cpu, write pytorch gpu version
                n_nan = get_not_nan(y[:, :, 0])  # LT10/50/90 not NAN
                loss_lt_10 = criterion(
                    out_lt_10[n_nan[0], n_nan[1]], y_torch[:, :, 0][n_nan[0], n_nan[1]][:, None])  # LT10 GT

                n_nan = get_not_nan(y[:, :, 1])  # LT10/50/90 not NAN
                loss_lt_50 = criterion(
                    out_lt_50[n_nan[0], n_nan[1]], y_torch[:, :, 1][n_nan[0], n_nan[1]][:, None])  # LT50 GT

                n_nan = get_not_nan(y[:, :, 2])  # LT10/50/90 not NAN
                loss_lt_90 = criterion(
                    out_lt_90[n_nan[0], n_nan[1]], y_torch[:, :, 2][n_nan[0], n_nan[1]][:, None])  # LT90 GT

                n_nan = get_not_nan(y[:, :, 3])  # budbreak not NAN
                loss_ph = criterion2(
                    out_ph[n_nan[0], n_nan[1]], y_torch[:, :, 3][n_nan[0], n_nan[1]][:, None])  # Budbreak GT

                loss = loss_lt_10 + loss_lt_50 + loss_lt_90 + loss_ph

                total_loss += loss.item()

                tepoch.set_postfix(Val_Loss=total_loss / count)
                tepoch.update(1)

            writer.add_scalar('Val_Loss', total_loss / count, epoch)
with torch.no_grad():
    out_lt_10, out_lt_50, out_lt_90, out_ph, _ = model(x_torch)
#out_lt = out_lt.numpy()
#out_ph = out_ph.numpy()

print(out_lt_10.shape, out_lt_50.shape, out_lt_90.shape, out_ph.shape)
# print(out_lt)
# print(out_ph)
modelSavePath = "./models/"
torch.save(model.state_dict(), modelSavePath + runID + ".pt")

### Plotting Functions 

In [ ]:
# plots residual LT to ground-truth
def graph_residual(plot_dict, title, x, xlabel, ylabel):

    plt.figure(figsize=(16, 9))

    x = np.array(x)
    width = 1
    pos = 0
    for key, value in plot_dict.items():

        plt.bar(x + pos, value, width, label=key)
        pos += width

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    plt.legend(loc='best')
    # plt.show()
    return

# plots residual LT to ground-truth
def graph_residual_lte(plot_dict, title, x, xlabel, ylabel):

    plt.figure(figsize=(16,9))

    x = np.array(x)
    width = 1
    pos = 0
    for key, value in plot_dict.items():
        
        plt.bar(x + pos, value, width, label = key)
        pos += width
    
    plt.ylim(-4, 4)
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)

    plt.legend(loc='best')
    plt.show()
    return
def graph_residual_ferg(title, ferg_diff, results_diff):
    ferg_diff = ferg_diff.flatten()
    results_diff = results_diff.flatten()
    
    days = np.arange(0, 252)
    plot_dict = {
                "Furguson": ferg_diff,
                "knn": results_diff,
                }
    graph_residual_lte(plot_dict, title, days, "Days", "Temperature. (Deg. C)")
    return
def graph_residual_tempdiff(title, results_diff):
    results_diff = results_diff.flatten()
    
    days = np.arange(0, 252)
    plot_dict = {
                "knn": results_diff,
                }
    graph_residual_lte(plot_dict, title, days, "Days", "Temperature. (Deg. C)")
    return

### Get predictions of Fergusen Model

In [ ]:
ferg_model = []

for i, season in enumerate(seasons):
    if i == 23 or i == 24:  # last 2 seasons for testing
        _y = df[["PREDICTED_Hc"]].loc[season, :].to_numpy()

        add_array = np.zeros((season_max_length - len(season), 1))
        add_array[:] = np.NaN

        _y = np.concatenate((_y, add_array), axis=0)

        ferg_model.append(_y)

print("ferg 2020", ferg_model[0].shape)
print("ferg 2021", ferg_model[1].shape)

### Plot RNN results

In [ ]:
gt = y_test.cpu().detach().numpy()
gt_2020_lt10 = gt[0, :, 0]
gt_2021_lt10 = gt[1, :, 0]

gt_2020_lt50 = gt[0, :, 1]
gt_2021_lt50 = gt[1, :, 1]

gt_2020_lt90 = gt[0, :, 2]
gt_2021_lt90 = gt[1, :, 2]

print("gt 2020 LT10", gt_2020_lt10.shape)
print("gt 2021 LT10", gt_2021_lt10.shape)

print("gt 2020 LT50", gt_2020_lt50.shape)
print("gt 2021 LT50", gt_2021_lt50.shape)

print("gt 2020 LT90", gt_2020_lt90.shape)
print("gt 2021 LT90", gt_2021_lt90.shape)

results_2020_lt10 = out_lt_10[0].cpu().numpy().flatten()
results_2021_lt10 = out_lt_10[1].cpu().numpy().flatten()
print("results_2021_lt10.shape", results_2021_lt10.shape)

results_2020_lt50 = out_lt_50[0].cpu().numpy().flatten()
results_2021_lt50 = out_lt_50[1].cpu().numpy().flatten()
print("results_2021_lt50.shape", results_2021_lt50.shape)

results_2020_lt90 = out_lt_90[0].cpu().numpy().flatten()
results_2021_lt90 = out_lt_90[1].cpu().numpy().flatten()
print("results_2021_lt90.shape", results_2021_lt90.shape)

ferg_diff_2020 = gt_2020_lt50 - ferg_model[0].flatten()
results_diff_2020_lt50 = gt_2020_lt50 - results_2020_lt50

print(ferg_diff_2020.shape)
print(results_diff_2020_lt50.shape)

ferg_diff_2020 = ferg_diff_2020.flatten()
results_diff_2020_lt50 = results_diff_2020_lt50.flatten()

days = np.arange(0, 252)
plot_dict = {
    str(runID): (gt_2020_lt10 - results_2020_lt10).flatten(),
}
graph_residual(plot_dict, "LT10 Same Day | 2020-2021 Season |  GT - Pred",
               days, "Days", "Temperature. (Deg. C)")

days = np.arange(0, 252)
plot_dict = {
    str(runID): (gt_2021_lt10 - results_2021_lt10).flatten(),
}
graph_residual(plot_dict, "LT10 Same Day | 2021 - * Season |  GT - Pred",
               days, "Days", "Temperature. (Deg. C)")

days = np.arange(0, 252)
plot_dict = {
    "Furguson": ferg_diff_2020,
    str(runID): results_diff_2020_lt50,
}
graph_residual(plot_dict, "LT50 Same Day | 2020-2021 Season |  GT - Pred",
               days, "Days", "Temperature. (Deg. C)")

ferg_diff_2021 = gt_2021_lt50 - ferg_model[1].flatten()
results_diff_2021_lt50 = gt_2021_lt50 - results_2021_lt50

# print(ferg_diff_2021.shape)
# print(results_diff_2021_lt50.shape)

ferg_diff_2021 = ferg_diff_2021.flatten()
results_diff_2021_lt50 = results_diff_2021_lt50.flatten()

days = np.arange(0, 252)
plot_dict = {
    "Furguson": ferg_diff_2021,
    str(runID): results_diff_2021_lt50,
}
graph_residual(
    plot_dict, "LT50 Same Day | 2021 - * Season |  GT - Pred", days, "Days", "LT50")

days = np.arange(0, 252)
plot_dict = {
    str(runID): (gt_2020_lt90 - results_2020_lt90).flatten(),
}
graph_residual(plot_dict, "LT90 Same Day | 2020-2021 Season |  GT - Pred",
               days, "Days", "Temperature. (Deg. C)")

days = np.arange(0, 252)
plot_dict = {
    str(runID): (gt_2021_lt90 - results_2021_lt90).flatten(),
}
graph_residual(plot_dict, "LT90 Same Day | 2021 - * Season |  GT - Pred",
               days, "Days", "Temperature. (Deg. C)")

### budbreak results

In [ ]:
bb_gt_2020_lt50 = gt[0, :, 1]
bb_gt_2021_lt50 = gt[1, :, 1]

bb_results_2020 = out_ph[0].cpu().numpy().flatten()
bb_results_2021 = out_ph[1].cpu().numpy().flatten()

#bb_results_2020 = np.where(bb_results_2020 > 0.5, 1, 0)
#bb_results_2021 = np.where(bb_results_2021 > 0.5, 1, 0)

plot_budbreak("Budbreak 2021-2022", seasons[-2], bb_results_2020)
plot_budbreak("Budbreak 2022-*", seasons[-1], bb_results_2021)

ferg_2020_under_rate = _get_under(ferg_model[0].flatten(), gt_2020_lt50)
print("ferg_2020 % gt < pred", ferg_2020_under_rate)

model_2020_under_rate = _get_under(results_2020_lt50, gt_2020_lt50)
print("model_2020 % gt < pred", model_2020_under_rate)

ferg_2021_under_rate = _get_under(ferg_model[1], gt_2021_lt50)
print("ferg_2021 % gt < pred", ferg_2021_under_rate)

model_2021_under_rate = _get_under(results_2021_lt50, gt_2021_lt50)
print("model_2021 % gt < pred", model_2021_under_rate)

### K-Nearest Neighbour Search

### data processing functions

In [ ]:
def pca_rdc(train_set, test_set, pca_thres=75, n_components=None):
    from sklearn.decomposition import PCA
    if n_components is None:
        pca = PCA()
        pca.fit(train_set)
        idx_reduction = pca.explained_variance_ratio_ >= np.percentile(
            pca.explained_variance_ratio_, pca_thres)
        n_components = idx_reduction.sum()

    pca = PCA(n_components=n_components)
    pca.fit(train_set)

    return pca.transform(train_set), pca.transform(test_set), n_components, pca


def get_sets(x, y, ox, model, test_set=False):
    with torch.no_grad():
        model.eval()
        pred_s, pred_rnn,  = model(x)

        if test_set == False:
            valuable_idx = get_not_nan(y)

            gt = y[valuable_idx[0], valuable_idx[1]]  # ground truth

            # penultimate layer before the last linear layer
            _penultimate = pred_s[valuable_idx[0], valuable_idx[1]]

            _rnn = pred_rnn[valuable_idx[0],
                            valuable_idx[1]]  # rnn output layer
        else:
            gt = y.view(-1)

            _penultimate = pred_s.reshape(
                pred_s.size(0) * pred_s.size(1), pred_s.size(2))

            _rnn = pred_rnn.reshape(pred_rnn.size(
                0) * pred_rnn.size(1), pred_rnn.size(2))

        return gt, _rnn, _penultimate

def IDW_KNN(train_x, train_gt, test_x, test_gt=None, n_neighbors=17, inverse_exp=-1, criterion=None):
    from sklearn.neighbors import NearestNeighbors
    # print(train_x.shape)

    nbrs = NearestNeighbors(n_neighbors=n_neighbors,
                            leaf_size=100).fit(train_x)
    pred_distances, pred_indices = nbrs.kneighbors(test_x)

    inverse_dist = pred_distances ** inverse_exp
    # print(inverse_dist.shape)
    norm_inverse_dist = inverse_dist / inverse_dist.sum(axis=1)[:, None]

    weigted_value = torch.squeeze(train_gt[pred_indices]) * norm_inverse_dist
    prediction = weigted_value.sum(axis=1).float()

    if (test_gt != None):
        loss = criterion(prediction, test_gt).item()
    else:
        loss = None

    return prediction, loss, norm_inverse_dist, nbrs

def knn_param_search(train_penul, test_penul, train_rnn, test_rnn, train_gt, test_gt, criterion, name=None):
    lowest_penul = float('inf')
    lowest_rnn = float('inf')

    k_e_penul = (None, None, None, None)
    k_e_rnn = (None, None, None, None)

    best_penul_knn = None

    best_rnn_knn = None

    best_penul_pca = None

    best_rnn_pca = None

    for pca_thres in [30, 50, 75, 100]:
        print("===================pca_thres: {}=====================".format(pca_thres))
        for j in range(-1, -6, -1):
            tl = []
            tr = []
            x_axis = []

            train_x_um_penul_rdc, test_x_um_penul_rdc, n_components_penul, pca_penul = pca_rdc(
                train_penul, test_penul, pca_thres=pca_thres)

            train_x_um_rnn_rdc, test_x_um_rnn_rdc, n_components_rnn, pca_rnn = pca_rdc(
                train_rnn, test_rnn, pca_thres=pca_thres)

            for i in range(2, 600, 5):

                # 1) penultimate layer before the last linear layer
                _, loss_penul, _, knn_penul = IDW_KNN(
                    train_x_um_penul_rdc, train_gt, test_x_um_penul_rdc, test_gt, n_neighbors=i, inverse_exp=j, criterion=criterion)

                # 2) RNN output
                _, loss_rnn, _, knn_rnn = IDW_KNN(
                    train_x_um_rnn_rdc, train_gt, test_x_um_rnn_rdc, test_gt, n_neighbors=i, inverse_exp=j, criterion=criterion)

                tl.append(loss_penul)
                tr.append(loss_rnn)
                x_axis.append(i)

                if loss_penul < lowest_penul:
                    k_e_penul = (i, j, pca_thres, n_components_penul)
                    lowest_penul = loss_penul
                    best_penul_knn = knn_penul
                    best_penul_pca = pca_penul
                    print("Loss / k_e_penul", lowest_penul, k_e_penul)

                if loss_rnn < lowest_rnn:
                    k_e_rnn = (i, j, pca_thres, n_components_rnn)
                    lowest_rnn = loss_rnn
                    best_rnn_knn = knn_rnn
                    best_rnn_pca = pca_rnn
                    print("Loss / k_e_rnn", lowest_rnn, k_e_rnn)

            # print(tl)
            #_params = "(n:" + str(i) + ", inverse_exp:" + str(j) + ")"
            # print("*********{}*********".format(j))
            # plt.cla()
            #plt.plot(x_axis, tl, label = "last")
            # plt.xlabel("K")
            # plt.ylabel("loss")
            #plt.title("Penultimate " + _params)
            # plt.show()
            # plt.cla()
            #plt.plot(x_axis, tr, label = "rnn")
            # plt.xlabel("K")
            # plt.ylabel("loss")
            #plt.title("RNN "+ _params)
            # plt.show()
            # plt.cla()

    # Save best to pkl

    best_penul = {
        "model": model_path,
        "neighbors": k_e_penul[0],
        "exponent": k_e_penul[1],
        "pca_thres": k_e_penul[2],
        "components": k_e_penul[3],
        "loss": loss_penul,
        "train_gt": train_gt,
        "pca": best_penul_pca,
        "knn": best_penul_knn,
    }

    best_rnn = {
        "model": model_path,
        "neighbors": k_e_rnn[0],
        "exponent": k_e_rnn[1],
        "pca_thres": k_e_rnn[2],
        "components": k_e_rnn[3],
        "loss": lowest_rnn,
        "train_gt": train_gt,
        "pca": best_rnn_pca,
        "knn": best_rnn_knn,
    }

    print(best_penul)

    print(best_rnn)

    if (best_penul['loss'] < best_rnn['loss']):
        with open('./knn/' + name + '_penul.pkl', 'wb') as _f:
            pickle.dump(best_penul, _f)
    elif (best_rnn['loss'] < best_penul['loss']):
        with open('./knn/' + name + '_rnn.pkl', 'wb') as _f:
            pickle.dump(best_rnn, _f)

    return k_e_penul, k_e_rnn

def knn_from_pkl(knn_path, y):
    with open(knn_path, 'rb') as _f:
        knn = pickle.load(_f)
    
    y_pca = knn['pca'].transform(y)
    
    pred_distances, pred_indices = knn['knn'].kneighbors(y_pca)

    inverse_dist = pred_distances ** knn['exponent']

    norm_inverse_dist = inverse_dist / inverse_dist.sum(axis = 1)[:, None]

    weigted_value = torch.squeeze(knn['train_gt'][pred_indices]) * norm_inverse_dist
    
    prediction = weigted_value.sum(axis = 1).float()
    
    return prediction

In [ ]:
model_path = "./pytorch_models/78304.pt"
model = knn_net(np.array(x_train).shape[-1])
model.load_state_dict(torch.load(model_path))
# LT10
# LT 10 and LT50 have 1-1 matching GT
train_gt_lt10, train_rnn, train_penul = get_sets(
    x_train, y_train[:, :, 0], x_train, model)
print("train_gt_lt10, train_rnn, train_penul",
      train_gt_lt10.shape, train_rnn.shape, train_penul.shape)

test_gt_lt10, test_rnn, test_penul = get_sets(
    x_test, y_test[:, :, 0], x_test, model)
print("test_gt_lt10, test_rnn, test_penul",
      test_gt_lt10.shape, test_rnn.shape, test_penul.shape)

# LT50
# LT 10 and LT50 have 1-1 matching GT
train_gt_lt50, _, _ = get_sets(x_train, y_train[:, :, 1], x_train, model)
print("train_gt_lt50.shape", train_gt_lt50.shape)

test_gt_lt50, _, _ = get_sets(x_test, y_test[:, :, 1], x_test, model)
print("test_gt_lt50.shape", test_gt_lt50.shape)

# BudBreak
train_gt_bb, _, _ = get_sets(x_train, y_train[:, :, 3], x_train, model)
print("train_gt_bb.shape", train_gt_bb.shape)

test_gt_bb, _, _ = get_sets(x_test, y_test[:, :, 3], x_test, model)
print("test_gt_bb.shape", test_gt_bb.shape)

# LT10
k_e_penul_lt10, k_e_rnn_lt10 = knn_param_search(train_penul=train_penul, test_penul=test_penul, train_rnn=train_rnn,
                                                test_rnn=test_rnn, train_gt=train_gt_lt10, test_gt=test_gt_lt10, criterion=nn.MSELoss(), name="LT10")
print("k_e_penul_lt10", k_e_penul_lt10)
print("k_e_rnn_lt10", k_e_rnn_lt10)

# LT50
k_e_penul_lt50, k_e_rnn_lt50 = knn_param_search(train_penul=train_penul, test_penul=test_penul, train_rnn=train_rnn,
                                                test_rnn=test_rnn, train_gt=train_gt_lt50, test_gt=test_gt_lt50, criterion=nn.MSELoss(), name="LT50")
print("k_e_penul_lt50", k_e_penul_lt50)
print("k_e_rnn_lt50", k_e_rnn_lt50)

# LT90

# LT90 Merlot has fewer GT
train_gt_lt90, train_rnn_lt90, train_penul_lt90 = get_sets(
    x_train, y_train[:, :, 2], x_train, model)
print("train_gt_lt90.shape, train_rnn_lt90.shape, train_penul_lt90.shape",
      train_gt_lt90.shape, train_rnn_lt90.shape, train_penul_lt90.shape)

test_gt_lt90, test_rnn_lt90, test_penul_lt90 = get_sets(
    x_test, y_test[:, :, 2], x_test, model)
print("test_gt_lt90.shape, test_rnn_lt90.shape, test_penul_lt90.shape",
      test_gt_lt90.shape, test_rnn_lt90.shape, test_penul_lt90.shape)

k_e_penul_lt90, k_e_rnn_lt90 = knn_param_search(train_penul=train_penul_lt90, test_penul=test_penul_lt90, train_rnn=train_rnn_lt90,
                                                test_rnn=test_rnn_lt90, train_gt=train_gt_lt90, test_gt=test_gt_lt90, criterion=nn.MSELoss(), name="LT90")
print("k_e_penul_lt90", k_e_penul_lt90)
print("k_e_rnn_lt90", k_e_rnn_lt90)

# BudBreak
train_gt_bb, train_rnn_bb, train_penul_bb = get_sets(
    x_train, y_train[:, :, 3], x_train, model)
print("train_gt_bb.shape, train_rnn_bb.shape, train_penul_bb.shape",
      train_gt_bb.shape, train_rnn_bb.shape, train_penul_bb.shape)

test_gt_bb, test_rnn_bb, test_penul_bb = get_sets(
    x_test, y_test[:, :, 3], x_test, model)
print("test_gt_bb.shape, test_rnn_bb.shape, test_penul_bb.shape",
      test_gt_bb.shape, test_rnn_bb.shape, test_penul_bb.shape)

k_e_penul_bb, k_e_rnn_bb = knn_param_search(train_penul=train_penul_bb, test_penul=test_penul_bb, train_rnn=train_rnn_bb,
                                            test_rnn=test_rnn_bb, train_gt=train_gt_bb, test_gt=test_gt_bb, criterion=nn.MSELoss(), name="Budbreak")
print("k_e_penul_bb", k_e_penul_bb)
print("k_e_rnn_bb", k_e_rnn_bb)

print("FINAL")
print("clusters / exponent / PCA / n components")
print("k_e_penul_lt10 =", k_e_penul_lt10)
print("k_e_rnn_lt10 =", k_e_rnn_lt10)
print("k_e_penul_lt50 =", k_e_penul_lt50)
print("k_e_rnn_lt50 =", k_e_rnn_lt50)
print("k_e_penul_lt90 =", k_e_penul_lt90)
print("k_e_rnn_lt90 =", k_e_rnn_lt90)
print("k_e_penul_bb =", k_e_penul_bb)
print("k_e_rnn_bb =", k_e_rnn_bb)
_, test_rnn_bb_all, _ = get_sets(x_test, y_test[:, :, 3], x_test, model, test_set=True)
pred_bb = knn_from_pkl(budbreak_knn_path, test_rnn_bb_all)
print(pred_bb.shape)
results_2020_bb = pred_bb[:252].numpy()
results_2021_bb = pred_bb[252:].numpy()

print(results_2020_bb.shape)
print(results_2021_bb.shape)
plot_budbreak("KNN Budbreak 2021-2022", seasons[-2], results_2020_bb)
plot_budbreak("KNN Budbreak 2022-*", seasons[-1], results_2021_bb)
gt = y_test.cpu().detach().numpy()
gt_2020_lt10 = gt[0, :, 0]
gt_2021_lt10 = gt[1, :, 0]

gt_2020_lt50 = gt[0, :, 1]
gt_2021_lt50 = gt[1, :, 1]

gt_2020_lt90 = gt[0, :, 2]
gt_2021_lt90 = gt[1, :, 2]

print("gt 2020 LT10", gt_2020_lt10.shape)
print("gt 2021 LT10", gt_2021_lt10.shape)

print("gt 2020 LT50", gt_2020_lt50.shape)
print("gt 2021 LT50", gt_2021_lt50.shape)

print("gt 2020 LT90", gt_2020_lt90.shape)
print("gt 2021 LT90", gt_2021_lt90.shape)

ferg_model = []

for i, season in enumerate(seasons):
    if i == 23 or i == 24: # last 2 seasons for testing
        _y = df[["PREDICTED_Hc"]].loc[season, :].to_numpy()
        
        add_array = np.zeros((season_max_length - len(season), 1))
        add_array[:] = np.NaN

        _y = np.concatenate((_y, add_array), axis=0)
        
        ferg_model.append(_y)

print("ferg 2020", ferg_model[0].shape)
print("ferg 2021", ferg_model[1].shape)
_, test_rnn_lt10_all, _ = get_sets(x_test, y_test[:, :, 0], x_test, model, test_set=True)
pred_lt10 = knn_from_pkl(lt10_knn_path, test_rnn_lt10_all)
print(pred_lt10.shape)
results_2020_lt10 = pred_lt10[:252].numpy()
results_2021_lt10 = pred_lt10[252:].numpy()

print(results_2020_lt10.shape)
print(results_2020_lt10.shape)
graph_residual_tempdiff("KNN LT10 Same Day | 2020-2021 Season |  GT - Pred", gt_2020_lt10 - results_2020_lt10)
graph_residual_tempdiff("KNN LT10 Same Day | 2021-* Season |  GT - Pred", gt_2021_lt10 - results_2020_lt10)
_, test_rnn_lt50_all, _ = get_sets(x_test, y_test[:, :, 1], x_test, model, test_set=True)
pred_lt50 = knn_from_pkl(lt50_knn_path, test_rnn_lt50_all)
print(pred_lt10.shape)
results_2020_lt50 = pred_lt50[:252].numpy()
results_2021_lt50 = pred_lt50[252:].numpy()

print(results_2020_lt50.shape)
print(results_2021_lt50.shape)
graph_residual_ferg("KNN LT50 Same Day | 2020-2021 Season |  GT - Pred", gt_2020_lt50 - ferg_model[0].flatten(), gt_2020_lt50 - results_2020_lt50)
graph_residual_ferg("KNN LT50 Same Day | 2021-* Season |  GT - Pred", gt_2021_lt50 - ferg_model[1].flatten(), gt_2021_lt50 - results_2021_lt50)
_, test_rnn_lt90_all, _ = get_sets(x_test, y_test[:, :, 2], x_test, model, test_set=True)
pred_lt90 = knn_from_pkl(lt90_knn_path, test_rnn_lt90_all)

print(pred_lt90.shape)
results_2020_lt90 = pred_lt90[:252].numpy()

results_2021_lt90 = pred_lt90[252:].numpy()

print(results_2020_lt90.shape)
print(results_2020_lt90.shape)
graph_residual_tempdiff("KNN lt90 Same Day | 2020-2021 Season |  GT - Pred", gt_2020_lt90 - results_2020_lt90)
graph_residual_tempdiff("KNN lt90 Same Day | 2021-* Season |  GT - Pred", gt_2021_lt90 - results_2020_lt90)